# Table of Contents

1. [Select Stocks](#select-stocks)
2. [Scrapling](#scrapling)
3. [Yahoo API]()
4. [Selenium](#selenium)
5. [Playwright](#playwright)
6. [Perplexity Results](#perplexity-results-gpt-4o)
   - 3.1 [Prompt](#prompt)
   - 3.2 [Response](#response)
   - 3.3 [Cointegration](#cointegration)
   - 3.4 [Correlation](#correlation)
7. [Cointegration](#cointegration)
8. [Sentiment](#sentiment)
9. [Trade Results](#trade-results)
10. [Overall Summary](#overall-summary)
11. [Breakdown Summary](#breakdown-summary)
12. [Future Works](#future-works)


# POC

Methodology for POC:

1. Choose "AAPL".
2. Scrape news (from Yahoo Finance), for Apple stock via scrapling
3. Use ChatGPT to determine the sentiment of each news on scale of 1 to 5:

- 1 -> Negative
- 2 -> Moderate negative
- 3 -> Neutral
- 4 -> Moderate positive
- 5 -> Positive

4. Use ChatGPT to determine the stock that has highest cointegration with Apple

5. Use Selenium to load more news article


In [2]:
import pandas as pd
import random
import sys
from pathlib import Path
import scrapling
from scrapling import Fetcher
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import yfinance as yf
import numpy as np
from collections import Counter
import math
from pprint import pformat
from decimal import Decimal
from scipy import stats

repo_dir = Path.cwd().as_posix()

if repo_dir not in sys.path:
    sys.path.append(repo_dir)

%load_ext autoreload
%autoreload 2

from src.utils import utils, yahoo_utils
from src.gen_data import GenData
from src.sentiment import SentimentRater
# from src.cointegrate import CoIntegrate
from src.gen_price_action import GenPriceAction
from src.cal_profit_loss import StockTrade
from src.cal_profit_loss import CalProfitLoss
from src.analyze_coint import AnalyzeCoint


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# [Select Stocks](#table-of-contents)

- Test "AAPL", "NVDA", and "PG" for POC.
- If time permits, then test the randomly selected stocks for each GICS Sector: ['PWR', 'INCY', 'CDW', 'CEG', 'BAC', 'LIN', 'BWA', 'ARE', 'CHTR', 'KDP', 'PSX']


In [33]:
stock_list = utils.gen_stock_list()
stock_list

{'Industrials': 'PWR',
 'Health Care': 'INCY',
 'Information Technology': 'CDW',
 'Utilities': 'CEG',
 'Financials': 'BAC',
 'Materials': 'LIN',
 'Consumer Discretionary': 'BWA',
 'Real Estate': 'ARE',
 'Communication Services': 'CHTR',
 'Consumer Staples': 'KDP',
 'Energy': 'PSX'}

In [36]:
utils.get_gics_sector(
    ["AAPL", "MSFT", "NVDA", "GOOGL", "META", "XOM", "CRM", "HD", "PG"]
)

{'AAPL': 'Information Technology',
 'MSFT': 'Information Technology',
 'NVDA': 'Information Technology',
 'GOOGL': 'Communication Services',
 'META': 'Communication Services',
 'XOM': 'Energy',
 'CRM': 'Information Technology',
 'HD': 'Consumer Discretionary',
 'PG': 'Consumer Staples'}

# [Scrapling](#table-of-contents)

- https://finance.yahoo.com/quote/AAPL/news/
- Scrapling aren't able to web scrap fully since the news are loaded when user scroll to the end of screen.


In [98]:
# Extract news related to AAPL from Yahoo Finance
news_info = yahoo_utils.extract_news_briefs(
    "https://finance.yahoo.com/quote/AAPL/news/"
)
news_info

[2025-03-08 21:01:29] INFO: Fetched (200) <GET https://finance.yahoo.com/quote/AAPL/news/> (referer: https://www.google.com/search?q=yahoo)


[{'publisher': 'Motley Fool',
  'period': datetime.timedelta(seconds=7200),
  'title': 'Could Apple Be a Millionaire-Maker Stock?',
  'content': 'During its rise to become a dominant tech enterprise, Apple (NASDAQ: AAPL) has certainly made its early investors some serious money. In the past two decades, shares have generated a total return of greater than 18,000%, boosted by the introduction of popular hardware devices and services. Could Apple be a millionaire-maker stock?'},
 {'publisher': 'Insider Monkey',
  'period': datetime.timedelta(seconds=43200),
  'title': 'Apple Inc. (AAPL) Unveils AI-Powered MacBook Air with M4 Chip',
  'content': 'We recently compiled a list of the 10 Hot AI News Updates Analysts are Monitoring. In this article, we are going to take a look at where Apple Inc. (NASDAQ:AAPL) stands against the other AI stocks. According to Goldman Sachs, the AI-powered rally in Chinese equities could boost other markets as well. Consequently, the firm has […]'},
 {'publisher

# [Yahoo API](#table-of-contents)

- Yahoo API gives very detailed news as info but limited to max 10 news article.


In [ ]:
ticker = yf.Ticker("AAPL")
ticker.news

[{'id': 'ee08c516-d812-48ab-9787-b73c34782bf7',
  'content': {'id': 'ee08c516-d812-48ab-9787-b73c34782bf7',
   'contentType': 'STORY',
   'title': "Wall Street's souring view on the US stock market isn't just about the economy — it's about AI too",
   'description': '',
   'summary': "As economic uncertainty has spawned a stock market sell-off, one of investors' favorite trades of the past two years is going through its own rerating of expectations.",
   'pubDate': '2025-03-11T17:44:49Z',
   'displayTime': '2025-03-11T17:44:49Z',
   'isHosted': True,
   'bypassModal': False,
   'previewUrl': None,
   'thumbnail': {'originalUrl': 'https://s.yimg.com/os/creatr-uploaded-images/2025-03/88bc1fd0-fb15-11ef-8dbd-ea4aea580f37',
    'originalWidth': 4500,
    'originalHeight': 3001,
    'caption': '',
    'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/CAOKVOjTmaybJSdSciq_vQ--~B/aD0zMDAxO3c9NDUwMDthcHBpZD15dGFjaHlvbg--/https://s.yimg.com/os/creatr-uploaded-images/2025-03/88bc1fd0-fb1

# [Selenium](#table-of-contents)

- Selenium is not suitable as it occasionally run into ReadTimeoutError causing the code to break. Hence we are not able to perform scrolling to required level.
- Running in Jupyter Notebook causes timeout error due to async operation of Jupyter Notebook.


In [ ]:
url = "https://finance.yahoo.com/quote/AAPL/news/"
scroll_limit = 5

driver = webdriver.Chrome()
driver.get(url)

# Locate <a> tag and navigate to its parent <div>
parent_div = driver.find_elements(By.XPATH, "//div[@class='content yf-82qtw3']")

# parent_div.get_attribute("class")

# # Print the class attribute of the parent <div>
# title_element = parent_div.find_element(By.XPATH, ".//h3[contains(@class, 'yf-82qtw3)]")
# title_element.text

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=37797): Read timed out. (read timeout=120)

# [Playwright](#table-of-contents)

- Able to perform scrolling to extract required number of articles as html content.
- Use BeautifulSoup on the generated html content to extract the relevant HTML tags.


In [ ]:
<div class="content yf-82qtw3"><a class="subtle-link fin-size-small titles noUnderline yf-1xqzjha"
        data-ylk="elm:hdln;elmt:link;itc:0;ct:story;slk:Shareholders%20are%20showing%20signs%20of%20DEI%20fatigue%20as%20activists%20push%20for%20more%20votes;sec:qsp-news;subsec:all;cpos:1;g:a42c9cba-3739-4dae-8402-e5acaa0ad944"
        href="https://finance.yahoo.com/news/shareholders-are-showing-signs-of-dei-fatigue-as-activists-push-for-more-votes-140136546.html"
        aria-label="Shareholders are showing signs of DEI fatigue as activists push for more votes"
        title="Shareholders are showing signs of DEI fatigue as activists push for more votes" data-rapid_p="23"
        data-v9y="1">
        <h3 class="clamp  yf-82qtw3">Shareholders are showing signs of DEI fatigue as activists push for more votes</h3>
        <p class="clamp  yf-82qtw3">Champions and critics of DEI policies are again pushing American companies to either
            bolster or diminish their diversity policies. But so far this year, none of the proposals are garnering
            shareholder support.</p>
    </a>
    <div class="footer yf-82qtw3">
        <div class="publishing yf-1weyqlp">Yahoo Finance <i aria-hidden="true" data-svelte-h="svelte-sg6bkb">•</i> 13
            hours ago</div>
        <div class="taxonomy-links yf-13tszyc"><a data-testid="ticker-container"
                class="ticker x-small hover2 border streaming yf-1fqyif7" aria-label="DE"
                data-ylk="elm:qte;elmt:link;itc:0;sec:qsp-news;slk:DE;subsec:all;cpos:1" href="/quote/DE/" title="DE"
                data-rapid_p="24" data-v9y="1">
                <div class="name yf-1fqyif7"><span class="symbol yf-1fqyif7">DE </span> </div> <fin-streamer
                    class="percentChange yf-1fqyif7" data-symbol="DE" data-field="regularMarketChangePercent"
                    data-trend="txt" data-pricehint="2" data-tstyle="default" active=""><span
                        class="d60f3b00 c4af00a5">+3.48%</span></fin-streamer>
            </a> <a data-testid="ticker-container" class="ticker x-small hover2 border streaming yf-1fqyif7"
                aria-label="COST" data-ylk="elm:qte;elmt:link;itc:0;sec:qsp-news;slk:COST;subsec:all;cpos:1"
                href="/quote/COST/" title="COST" data-rapid_p="25" data-v9y="1">
                <div class="name yf-1fqyif7"><span class="symbol yf-1fqyif7">COST </span> </div> <fin-streamer
                    class="percentChange yf-1fqyif7" data-symbol="COST" data-field="regularMarketChangePercent"
                    data-trend="txt" data-pricehint="2" data-tstyle="default" active=""><span
                        class="d60f3b00 d5882186">-6.07%</span></fin-streamer>
            </a> <a data-testid="ticker-container" class="ticker x-small hover2 border streaming yf-1fqyif7"
                aria-label="AAPL" data-ylk="elm:qte;elmt:link;itc:0;sec:qsp-news;slk:AAPL;subsec:all;cpos:1"
                href="/quote/AAPL/" title="AAPL" data-rapid_p="26" data-v9y="1">
                <div class="name yf-1fqyif7"><span class="symbol yf-1fqyif7">AAPL </span> </div> <fin-streamer
                    class="percentChange yf-1fqyif7" data-symbol="AAPL" data-field="regularMarketChangePercent"
                    data-trend="txt" data-pricehint="2" data-tstyle="default" active=""><span
                        class="d60f3b00 c4af00a5">+1.59%</span></fin-streamer>
            </a> </div>
    </div>
</div>

# [Perplexity Results (GPT-4o)](#table-of-contents)


## [Prompt](#table-of-contents)

The list of dictionaries are attached with the following prompt in Perplexity:

```
You are a stock analysis expert.

I've attached list of dictionaries containing news info specifically:

1. publisher = Source where news are generated
2. period = Amount of time lapsed after news release
3. title = News title
4. content = News content in brief

Each dictionary is supposed to be related to Apple stock.

Your task:
1. Review the 'title' and 'content' keys in each dictionary to access the sentiment related to Apple stock.
2. Use ONLY following rating:

    1 = Negative
    2 = Moderate negative
    3 = Neutral
    4 = Moderate positive
    5 = Positive

3. If the news are not related to a Apple stock, please give a rating of 3.
4. Please review all dictionary items in the attached list and provide the corresponding rating.
5. After reviewing all dictionary items, kindly provide an overall sentiment rating for Apple stock.
```


## [Response](#table-of-contents)

Response from Perplexity (ChatGPT 4-o):

```
Sentiment Ratings for Each News Item
Title: "Could Apple Be a Millionaire-Maker Stock?"
Content: Highlights Apple's historical success and potential for future gains.
Rating: 5 (Positive)

Title: "Apple Inc. (AAPL) Unveils AI-Powered MacBook Air with M4 Chip"
Content: Discusses the launch of an innovative product, indicating progress in AI.
Rating: 5 (Positive)

Title: "Apple (AAPL) Beats Stock Market Upswing: What Investors Need to Know"
Content: Reports a positive trading session for Apple stock.
Rating: 4 (Moderate positive)

Title: "Apple Delays Siri Upgrade Indefinitely as AI Concerns Escalate"
Content: Focuses on delays in Apple's AI projects, raising concerns.
Rating: 2 (Moderate negative)

Title: "Apple Is Pushing Back AI-Driven Siri Features to 2026, Report Says"
Content: Similar to the previous item, emphasizes delays in Siri enhancements.
Rating: 2 (Moderate negative)

Title: "Apple Says Some AI-Powered Enhancements to Siri to Be Delayed"
Content: Mentions delays in Siri updates amid slumping iPhone sales.
Rating: 2 (Moderate negative)

Title: "Sector Update: Tech Stocks Advance Late Afternoon"
Content: General update on tech stocks without specific focus on Apple.
Rating: 3 (Neutral)

Title: "Magnificent Seven Stocks: Nvidia Reverses Higher; Amazon, Meta, Tesla Slide"
Content: Discusses multiple tech stocks, with a brief mention of Apple’s mixed performance in 2025.
Rating: 3 (Neutral)

Title: "Apple says some AI improvements to Siri delayed to 2026"
Content: Reiterates delays in Siri's AI features, consistent with earlier reports.
Rating: 2 (Moderate negative)

Title: "Apple's First Foldable iPhone Could Launch in 2026 With Premium Pricing and AI Features"
Content: Speculates on a high-priced foldable iPhone with advanced features, suggesting innovation but no immediate impact.
Rating: 4 (Moderate positive)

Title: "Apple Inc. (AAPL): Jim Cramer Cautions – ‘I Could See the Stock Going Lower!’"
Content: Warns of potential downside for Apple stock, reflecting negativity.
Rating: 1 (Negative)

Title: "Apple Secures Local Content Certifications in Indonesia but Faces Additional Regulatory Hurdles"
Content: Notes progress in regulatory compliance but highlights challenges ahead.
Rating: 3 (Neutral)

Title: "Is Apple Inc. (AAPL) Among the Best Warren Buffett Stock to Buy Right Now?"
Content: Discusses Apple's position as a strong investment option under Warren Buffett’s philosophy, implying positivity.
Rating: 4 (Moderate positive)

Title: "The Mag 7 Stocks Are No Longer Magnificent. Where the Charts Say They Go Next."
Content: Broad commentary on tech stocks, including Apple, with no clear sentiment focus on Apple itself.
Rating: 3 (Neutral)

Title: "Broadcom Shares Jump as AI Growth Fuels Upbeat Forecast"
Content: Primarily about Broadcom, with indirect relevance to Apple as a client company.
Rating: 3 (Neutral)

Title: "Major Breakthrough: New Certificates Propel Apple iPhone 16 into Indonesia"
Content: Highlights progress in regulatory certification for iPhone sales in Indonesia, indicating positive developments for Apple’s market expansion.
Rating: 4 (Moderate positive)

Title: "In Rotten Market, Top Funds Harvest Apple Stock — And 10 Others"
Content: Indicates institutional confidence in Apple stock despite broader market challenges, reflecting positivity.
Rating: 4 (Moderate positive)

Title: "Apple's iPhone 16 Still on Hold in Indonesia Despite Key Certification Win"
Content: Acknowledges regulatory progress but emphasizes ongoing hurdles for product launch in Indonesia.
Rating: 3 (Neutral)

Title: "Cloud AI Today - Cirrascale Launches Inference Cloud For Scalable AI Integration"
Content: Not related to Apple stock specifically but mentions general AI developments in the industry.
Rating: 3 (Neutral)

Title: "38% of Berkshire Hathaway's Portfolio Is Invested in These 3 Unstoppable Dividend Stocks"
Content: General commentary on Warren Buffett’s portfolio with no specific focus on Apple stock sentiment.
Rating: 3 (Neutral)

Overall Sentiment Rating for Apple Stock
The individual ratings suggest a mix of sentiments:

Positive/Moderate Positive Ratings (5 or 4): 8 items

Neutral Ratings (3): 8 items

Negative/Moderate Negative Ratings (1 or 2): 4 items

The overall sentiment leans toward moderately positive for Apple stock, reflecting optimism tempered by some concerns over delays and market challenges.

Final Overall Sentiment Rating:
4 (Moderate positive)
```


## [Cointegration](#table-of-contents)

ChatGPT is not able to provide the stock with highest cointegration value with Apple stock:

```
The query about which S&P 500 stock has the highest cointegration value with Apple stock cannot be answered directly using the provided search results. Cointegration analysis requires statistical computations based on historical price data of Apple stock and other S&P 500 stocks. This involves time-series analysis to determine the long-term equilibrium relationship between Apple and other stocks.

If you have access to historical price data for Apple and other S&P 500 stocks, you can calculate cointegration values using statistical software or programming languages such as Python or R. The process typically involves:

Collecting Data: Obtain historical price data for Apple (AAPL) and other S&P 500 stocks over a specific period.

Stationarity Testing: Use tests like the Augmented Dickey-Fuller (ADF) test to ensure the time series are stationary.

Cointegration Testing: Apply tests such as the Engle-Granger two-step method or Johansen test to identify stocks with high cointegration values with Apple.

If you provide relevant price data or specify tools available, I can guide you further on how to perform this analysis.
```


## [Correlation](#table-of-contents)

ChatGPT 4-o stated Microsoft has the highest degree of correlation:

```
The stock most correlated with Apple (AAPL) among S&P 500 companies is Microsoft (MSFT). Historically, Apple and Microsoft have shown a high degree of correlation due to their positions as leading technology companies and their exposure to similar market dynamics, such as consumer electronics, cloud computing, and software services.

While specific numerical correlation values for S&P 500 stocks are not provided in the search results, Microsoft is widely recognized as one of Apple’s closest peers in terms of performance trends and market behavior. If you need precise correlation coefficients, you would need to calculate them using historical price data for Apple and other S&P 500 stocks over a specific period. Tools like Python, R, or financial platforms can help generate a correlation matrix for this purpose.
```


# [Compute Cointegration](#table-of-contents)

A number of issues with downloading OHLCV data via yfinance:

1. API misuse happen randomly dispite rate limtting and caching.
2. Inaccurate download of OHLCV data using yf.download(<ticker_list>) e.g.

- ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A'] -> ABBV not downloaded but ABNB downloaded instead.
- ABNB downloaded twice.

As such, we have to resort to downloading individual ticker separately. Note that yfinance do not have OHLCV data for 'CTAS', 'BRK.B' and 'BF.B' tickers; therefore they are under the ignored_list.


In [4]:
co_integrate = CoIntegrate()
co_integrate.run()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


    1) MMM    : 1305



[*********************100%***********************]  1 of 1 completed


    2) AOS    : 1305



...

  499) ZBRA   : 1305



[*********************100%***********************]  1 of 1 completed


  500) ZBH    : 1305



[*********************100%***********************]  1 of 1 completed

  501) ZTS    : 1305



In [ ]:
# 'CTAS' OHLCV dta is not available in yfinance
yf.download("CTAS", start="2020-01-01")

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['CTAS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2025-03-13)')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,CTAS,CTAS,CTAS,CTAS,CTAS,CTAS
Date,,,,,,


In [4]:
# Cointegration stocks with 'AAPL' ticker
df_28 = utils.get_coint_ticker("AAPL", "2025-03-28")
df_28

,ticker1,ticker2,pvalue,coint_t,percent_1,percent_5,percent_10,cointegrate
18170,AAPL,PG,0.000914,-4.576653,-3.905181,-3.340999,-3.047828,1
18039,AAPL,INCY,0.005623,-4.073665,-3.905181,-3.340999,-3.047828,1
18121,AAPL,MSI,0.006085,-4.050084,-3.905181,-3.340999,-3.047828,1
18208,AAPL,SO,0.006984,-4.008461,-3.905181,-3.340999,-3.047828,1
18148,AAPL,ORCL,0.007844,-3.972982,-3.905181,-3.340999,-3.047828,1
17845,AAPL,ANET,0.008812,-3.937046,-3.905181,-3.340999,-3.047828,1
18022,AAPL,HPE,0.009419,-3.916261,-3.905181,-3.340999,-3.047828,1
18152,AAPL,PANW,0.010237,-3.890069,-3.905181,-3.340999,-3.047828,1
18178,AAPL,PWR,0.010473,-3.882867,-3.905181,-3.340999,-3.047828,1
17944,AAPL,DG,0.011050,-3.865832,-3.905181,-3.340999,-3.047828,1


In [3]:
from scipy import stats

res = stats.spearmanr([1, 2, 3, 4, 5], [5, 6, 7, 8, 7])
res.statistic, res.pvalue

(np.float64(0.8207826816681233), np.float64(0.08858700531354381))

In [4]:
res = stats.pearsonr([1, 2, 3, 4, 5], [5, 6, 7, 8, 7])
res.statistic, res.pvalue

(np.float64(0.8320502943378436), np.float64(0.0805095732984986))

In [5]:
res = stats.kendalltau([1, 2, 3, 4, 5], [5, 6, 7, 8, 7])
res.statistic, res.pvalue

(np.float64(0.7378647873726218), np.float64(0.07697417298126674))

## Analyze AAPL


In [5]:
analyze_coint = AnalyzeCoint()
df_coint_analysis = analyze_coint("AAPL")
df_coint_analysis

,date1,period1,top_20_tickers1,date2,period2,top_20_tickers2,num_common,common_tickers
0,2025-03-13,1,"VLO, PSX, CSGP, COP, TJX, LII, MOS, MPC, EQIX,...",2025-03-13,3,"DVN, APA, OXY, VRSK, TYL, ORCL, SPGI, EL, LKQ,...",1,EQIX
1,2025-03-13,1,"VLO, PSX, CSGP, COP, TJX, LII, MOS, MPC, EQIX,...",2025-03-13,5,"PG, OKE, BKR, HPE, MAR, L, PEG, WFC, SPG, PPL,...",0,
2,2025-03-13,1,"VLO, PSX, CSGP, COP, TJX, LII, MOS, MPC, EQIX,...",2025-03-26,1,"LII, MOS, PSX, TJX, VLO, CSGP, ZBH, GEN, NVDA,...",8,"VLO, PSX, CSGP, COP, TJX, LII, MOS, ZBH"
3,2025-03-13,1,"VLO, PSX, CSGP, COP, TJX, LII, MOS, MPC, EQIX,...",2025-03-26,3,"APA, VRSK, ORCL, DVN, EL, WBA, TYL, OXY, ARE, ...",2,"MOS, EQIX"
4,2025-03-13,1,"VLO, PSX, CSGP, COP, TJX, LII, MOS, MPC, EQIX,...",2025-03-26,5,"PG, OKE, MSI, HRL, HPE, PANW, INCY, DG, ANET, ...",0,
5,2025-03-13,1,"VLO, PSX, CSGP, COP, TJX, LII, MOS, MPC, EQIX,...",2025-03-28,1,"MOS, TJX, LII, VLO, PSX, CSGP, KMX, GEN, ZBH, ...",7,"VLO, PSX, CSGP, TJX, LII, MOS, ZBH"
6,2025-03-13,1,"VLO, PSX, CSGP, COP, TJX, LII, MOS, MPC, EQIX,...",2025-03-28,3,"APA, EL, VRSK, WBA, ARE, DVN, ORCL, OXY, TYL, ...",0,
7,2025-03-13,1,"VLO, PSX, CSGP, COP, TJX, LII, MOS, MPC, EQIX,...",2025-03-28,5,"PG, INCY, MSI, SO, ORCL, ANET, HPE, PANW, PWR,...",0,
8,2025-03-13,3,"DVN, APA, OXY, VRSK, TYL, ORCL, SPGI, EL, LKQ,...",2025-03-13,5,"PG, OKE, BKR, HPE, MAR, L, PEG, WFC, SPG, PPL,...",0,
9,2025-03-13,3,"DVN, APA, OXY, VRSK, TYL, ORCL, SPGI, EL, LKQ,...",2025-03-26,1,"LII, MOS, PSX, TJX, VLO, CSGP, ZBH, GEN, NVDA,...",1,EL


In [18]:
df_coint_analysis.loc[
    df_coint_analysis["period1"] == df_coint_analysis["period2"], :
].sort_values(by=["date1", "period1"])

,date1,period1,top_20_tickers1,date2,period2,top_20_tickers2,num_common,common_tickers
28,2025-03-13,1,"VLO, PSX, CSGP, COP, TJX, LII, MOS, MPC, EQIX,...",2025-03-28,1,"MOS, TJX, LII, VLO, PSX, CSGP, KMX, GEN, ZBH, ...",7,"VLO, PSX, CSGP, TJX, LII, MOS, ZBH"
23,2025-03-13,3,"DVN, APA, OXY, VRSK, TYL, ORCL, SPGI, EL, LKQ,...",2025-03-28,3,"APA, EL, VRSK, WBA, ARE, DVN, ORCL, OXY, TYL, ...",15,"DVN, APA, OXY, VRSK, TYL, ORCL, SPGI, EL, MCO,..."
32,2025-03-13,5,"PG, OKE, BKR, HPE, MAR, L, PEG, WFC, SPG, PPL,...",2025-03-28,5,"PG, INCY, MSI, SO, ORCL, ANET, HPE, PANW, PWR,...",8,"PG, OKE, BKR, HPE, L, PEG, HLT, SO"
10,2025-03-26,1,"LII, MOS, PSX, TJX, VLO, CSGP, ZBH, GEN, NVDA,...",2025-03-13,1,"VLO, PSX, CSGP, COP, TJX, LII, MOS, MPC, EQIX,...",8,"LII, MOS, PSX, TJX, VLO, CSGP, ZBH, COP"
13,2025-03-26,1,"LII, MOS, PSX, TJX, VLO, CSGP, ZBH, GEN, NVDA,...",2025-03-28,1,"MOS, TJX, LII, VLO, PSX, CSGP, KMX, GEN, ZBH, ...",10,"LII, MOS, PSX, TJX, VLO, CSGP, ZBH, GEN, NVDA,..."
2,2025-03-26,3,"APA, VRSK, ORCL, DVN, EL, WBA, TYL, OXY, ARE, ...",2025-03-13,3,"DVN, APA, OXY, VRSK, TYL, ORCL, SPGI, EL, LKQ,...",15,"APA, VRSK, ORCL, DVN, EL, WBA, TYL, OXY, ANET,..."
5,2025-03-26,3,"APA, VRSK, ORCL, DVN, EL, WBA, TYL, OXY, ARE, ...",2025-03-28,3,"APA, EL, VRSK, WBA, ARE, DVN, ORCL, OXY, TYL, ...",18,"APA, VRSK, ORCL, DVN, EL, WBA, TYL, OXY, ARE, ..."
17,2025-03-26,5,"PG, OKE, MSI, HRL, HPE, PANW, INCY, DG, ANET, ...",2025-03-13,5,"PG, OKE, BKR, HPE, MAR, L, PEG, WFC, SPG, PPL,...",9,"PG, OKE, HPE, HLT, SO, BKR, L, MAR, PEG"
20,2025-03-26,5,"PG, OKE, MSI, HRL, HPE, PANW, INCY, DG, ANET, ...",2025-03-28,5,"PG, INCY, MSI, SO, ORCL, ANET, HPE, PANW, PWR,...",19,"PG, OKE, MSI, HRL, HPE, PANW, INCY, DG, ANET, ..."


# [Sentiment](#table-of-contents)

- Capture publication date, publisher (e.g. Bloomberg), news title and news content via yfinance news filtered by ticker e.g. https://finance.yahoo.com/quote/AAPL/news/
- Use different HuggingFace FinBERT on news title only, news content only and both news title and content:

| FinBERT                                                                                    | Model Card                            | Column      | Description                                                                                                                                                      |
| ------------------------------------------------------------------------------------------ | ------------------------------------- | ----------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| [Prosus AI](https://huggingface.co/ProsusAI/finbert)                                       | ProsusAI/finbert                      | prosusai    | &bull; Pre-trained NLP model to analyze sentiment of financial text. <br>&bull; Used Financial PhraseBank by Malo et al. (2014) is used for fine-tuning.         |
| [finbert-tone](https://huggingface.co/yiyanghkust/finbert-tone)                            | yiyanghkust/finbert-tone              | yiyanghkust | &bull; Pre-trained on corporate reports, earning calls, and analyst reports. <br>&bull; Fined-tuned on 10,000 manually annotated sentences from analyst reports. |
| [FinBERT-FOMC](https://huggingface.co/ZiweiChen/FinBERT-FOMC)                              | ZiweiChen/FinBERT-FOMC                | ziweichen   | &bull; Fine-tuned version of ProsusAI/finbert. <br>&bull; Fine-tuned on data useed in FOMC minutes from Jan 2006 to Feb 2023.                                    |
| [finbert-sentiment-analysis](https://huggingface.co/AventIQ-AI/finbert-sentiment-analysis) | AventIQ-AI/finbert-sentiment-analysis | aventiq_ai  | &bull; FinBERT model fine-tuned for sentiment analysis using English/Quotes dataset.                                                                             |

## Observations

- In general, FinBERT is able to rate the sentiment correctly to a certain extent.
- Different variants perform differently based on the fine-tuned dataset - 262 news (out of 3791) are rated as both negative and positive concurrently based on 28 March analysis.
- 'FinBERT-FOMC' appears to have more consistent rating among the 4 (Needs further investigations).


In [2]:
df = pd.read_csv("./data/sentiment.csv")
df

,pub_date,ticker,publisher,period,title,content,prosusai,prosusai_title,prosusai_content,yiyanghkust,yiyanghkust_title,yiyanghkust_content,ziweichen,ziweichen_title,ziweichen_content,aventiq_ai,aventiq_ai_title,aventiq_ai_content
0,2025-03-28 18:53:00,AAPL,Yahoo Finance,2 hours ago,Trump is set on tariffing chips. It’s not so s...,President Trump's plan to put 25% tariffs on c...,3,3,3,3,3,3,3,3,2,4,3,1
1,2025-03-28 20:08:00,AAPL,Simply Wall St.,45 minutes ago,Cloud AI Today - Boosting Performance Akamai a...,Fermyon Technologies and Akamai have announced...,5,5,5,5,5,5,5,5,5,5,5,5
2,2025-03-28 19:53:00,AAPL,MT Newswires,1 hour ago,"Market Chatter: Apple, Meta Platforms Could be...",Apple (AAPL) and Meta Platforms (META) could b...,2,1,3,3,3,3,2,2,2,1,1,2
3,2025-03-28 19:53:00,AAPL,Motley Fool,1 hour ago,Prediction: This Artificial Intelligence (AI) ...,The trillion-dollar club could be joined by an...,3,3,3,3,3,3,3,3,3,5,5,5
4,2025-03-28 18:53:00,AAPL,LA Times,2 hours ago,Expansive offices for Apple are rising in Culv...,Construction crews have erected cranes and are...,4,5,4,5,3,5,5,3,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3788,2025-03-04 21:02:00,BA,The Telegraph,24 days ago,British Airways opens new Gatwick hangar to cu...,British Airways is to open a new maintenance h...,4,4,3,3,3,3,3,3,3,5,2,5
3789,2025-03-03 21:02:00,BA,PA Media: Money,25 days ago,Aerospace giant Senior shrugs off Boeing strik...,"The UK firm, which is based in Rickmansworth, ...",1,2,1,2,1,3,1,3,3,1,1,1
3790,2025-03-03 21:02:00,BA,Reuters,25 days ago,Norwegian to buy 10 leased Boeing aircraft,COPENHAGEN (Reuters) -Budget carrier Norwegian...,5,3,5,5,3,5,3,3,3,5,5,5
3791,2025-03-01 21:02:00,BA,Motley Fool,27 days ago,Boeing's Space Business Could Be Stoppable Aft...,I was wrong about Boeing in December. Imagine ...,3,3,3,1,1,1,3,1,3,1,2,1


In [3]:
print(df["ticker"].nunique())
df["ticker"].unique()

19


array(['AAPL', 'NVDA', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'JPM',
       'JNJ', 'V', 'XOM', 'UNH', 'WMT', 'PG', 'HD', 'NFLX', 'CRM', 'BAC',
       'BA'], dtype=object)

In [ ]:
utils.display_divergent_rating(df)

divergent news : 262


,pub_date,ticker,publisher,period,title,content,prosusai,yiyanghkust,ziweichen,aventiq_ai
30,2025-03-27 07:40:00,AAPL,Motley Fool,yesterday,1 Unstoppable Vanguard ETF to Confidently Buy With $400 During the Stock Market Sell-Off,"The S&P 500 (SNPINDEX: ^GSPC) index returned 25% (including dividends) in 2024, which was more than double its long-term average of 10.5%. The Vanguard Growth ETF (NYSEMKT: VUG) is an exchange-traded fund that tracks the performance of the CRSP Growth Index by holding the same stocks and maintaining similar weightings, and it has outperformed the S&P 500 every year, on average, since it was established in 2004. The Vanguard ETF is currently down 11% from its record high amid the broader stock market sell-off, so investors can pick up a single share for under $400.",1,5,3,1
82,2025-03-23 07:40:00,AAPL,Insider Monkey,5 days ago,"Apple (AAPL) Poised for AI Boom Without Heavy GPU Investments, Says Evercore","We recently published a list of 10 High-Flying AI Stocks to Watch Today. In this article, we are going to take a look at where Apple Inc. (NASDAQ:AAPL) stands against other high-flying AI stocks to watch today. The most important conference in the tech world is in full swing, yet investors are skeptical about what’s […]",3,1,1,5
95,2025-03-22 07:40:00,AAPL,Zacks,6 days ago,Severance's Fame is Good for Apple TV+: Is it True for AAPL Stock?,"Severance's success bodes well for Apple's Services business. However, Apple Intelligence delays, stretched valuation, and China headwinds are concerns in 2025.",2,5,1,1
96,2025-03-22 07:40:00,AAPL,Medical Device Network,6 days ago,"Apple’s hearing aid software is impressive, but audiologists still matter",An expert from the Royal National Institute for Deaf People is enthusiastic about the innovation but cautions about its medical limitations.,2,5,3,1
120,2025-03-21 07:40:00,AAPL,GuruFocus.com,7 days ago,Apple's $1 Billion Streaming Sinkhole: Is This the Costliest Gamble in Tech?,Apple TV+ is bleeding cash while rivals dominate--yet investors remain unfazed. Is Apple quietly rewriting the streaming playbook?,1,5,2,1
125,2025-03-21 07:40:00,AAPL,GuruFocus.com,7 days ago,Apple TV+ Inches to 45M Users as Losses Reach $1B Yearly,Apple TV+ is nearing 45 million subscribers despite incurring approximately $1 billion in losses each year,2,4,5,1
134,2025-03-21 07:40:00,AAPL,Motley Fool,7 days ago,Will Apple's Slow Artificial Intelligence Rollout Help or Hurt the Stock?,"Artificial intelligence (AI) is a huge opportunity for a device maker like Apple (NASDAQ: AAPL) to take advantage of. With more than 2.2 billion active Apple devices in the world, if the company can use AI to help create a new income stream for its operations, that can be a huge growth catalyst for the business, which is something Apple needs given its underwhelming growth in recent years. While the company has announced plans for Apple Intelligence to run on its phones and for that to potentially lead to more sales, it's been lagging behind its rivals.",5,5,1,5
145,2025-03-20 07:40:00,AAPL,Yahoo Finance,8 days ago,Nvidia's GTC fallout reminds 'Magnificent 7' investors of an important lesson,"Nvidia is still the straw that stirs the drink for the ""Magnificent Seven.""",3,1,4,5
171,2025-03-19 07:40:00,AAPL,Insider Monkey,9 days ago,Apple(NASDAQ:AAPL) Bullish on AI Supercycle vs. Cautious on Current Progress,"Recently we took a detailed look at the Top 10 Trending Stocks and Apple Inc. (NASDAQ:AAPL) was one of the stocks that were highlighted in that article. The volatile tariff policies of the new US administration and a slowdown in AI-related enthusiasm took a toll on the market over the past few weeks. However, some analysts […]",3,5,2,1
179,2025-03-19 07:40:00,AAPL,Motley Fool,9 days ago,"Apple Partner Globalstar Just Set a New Record, but Did You Even Notice?","On Feb. 27, satellite communications company Globalstar (NASDAQ: GSAT) reported the end of its best revenue year ever. Two weeks after announcin

# [Trade Results](#table-of-contents)


In [105]:
results = utils.load_csv("./data/results/2025-03-28/trade_results.csv")
results

,ticker,coint_ticker,action,entry_date,entry_price,exit_date,exit_price,profit_loss,days_held,win
0,HD,MSI,buy,2025-01-28,464.02,2025-03-06,422.2,-41.82,37,0
1,HD,MSI,buy,2025-02-26,425.15,2025-03-06,422.2,-2.95,8,0
2,HD,MSI,buy,2025-02-27,428.42,2025-03-06,422.2,-6.22,7,0
3,HD,MSI,buy,2025-02-28,439.06,2025-03-06,422.2,-16.86,6,0
4,HD,MSI,buy,2025-03-03,426.23,2025-03-06,422.2,-4.03,3,0
...,...,...,...,...,...,...,...,...,...,...
1166,BA,CMCSA,buy,2025-03-12,35.19,2025-03-13,34.7,-0.49,1,0
1167,BA,CMCSA,buy,2025-03-14,35.43,2025-03-19,36.09,0.66,5,1
1168,BA,CMCSA,buy,2025-03-18,36.01,2025-03-19,36.09,0.08,1,1
1169,BA,CMCSA,buy,2025-03-20,36.5,2025-03-25,36.94,0.44,5,1


# [Overall Summary](#table-of-contents)


In [16]:
overall = utils.load_csv("./data/results/2025-03-28/overall_summary.csv", index_col=0)
overall

,Overall Statistics
stock_ticker_without_trades,AAPL
stock_ticker_with_trades,"HD, CRM, TSLA, AMZN, V, NVDA, BA, JNJ, PG, JPM..."
total_num_trades,1171
total_num_wins,597
total_num_loss,574
total_profit,-293.94
max_loss,-84.56
max_profit,32.61
mean_profit,-0.25101622544833474
median_profit,0.02


# [Breakdown Summary](#table-of-contents)


In [101]:
breakdown = utils.load_csv(
    "./data/results/2025-03-28/breakdown_summary.csv", header=[0, 1], index_col=[0, 1]
)
breakdown

entry_date               exit_date days_held            \
                            min         max         max       min max  mean   
ticker coint_ticker                                                           
AMZN   BAC           2025-03-20  2025-03-26  2025-03-27         1   7  4.25   
       CI            2025-03-20  2025-03-26  2025-03-27         1   7  4.25   
       HII           2025-03-20  2025-03-26  2025-03-27         1   7  4.25   
       HSY           2025-03-20  2025-03-26  2025-03-27         1   7  4.25   
       INVH          2025-03-20  2025-03-26  2025-03-27         1   7  4.25   
...                         ...         ...         ...       ...  ..   ...   
XOM    GOOG          2025-02-26  2025-03-26  2025-03-27         1  10   4.7   
       GOOGL         2025-02-26  2025-03-26  2025-03-27         1  10   4.7   
       SJM           2025-02-26  2025-03-26  2025-03-27         1  10   4.7   
       WSM           2025-02-26  2025-03-26  2025-03-27         1  10   4.7   
       WY            2025-02-26  2025-03-26  2025-03-27         1  10   4.7   

                           entry_price profit_loss                         \
                    median         sum         min    max    mean  median   
ticker coint_ticker                                                         
AMZN   BAC             4.5      170.84       -0.51   0.09   -0.15   -0.09   
       CI              4.5     1279.67        3.35   8.09  4.8225   3.925   
       HII             4.5      812.34       -2.74   1.68   0.065    0.66   
       HSY             4.5      668.18         2.1   6.15   3.575   3.025   
       INVH            4.5      136.52       -0.21   0.29    0.03    0.02   
...                    ...         ...         ...    ...     ...     ...   
XOM    GOOG            3.5     1675.28       -4.48   3.58  -0.896  -0.845   
       GOOGL           3.5     1654.83       -4.22   3.32  -0.953    -0.9   
       SJM             3.5     1114.87       -4.56   2.25  -0.901   -1.25   
       WSM             3.5     1752.93      -11.35  -0.94  -6.299   -6.47   
       WY              3.5      296.13       -0.93   1.28  -0.126   -0.19   

                              win     trading_period percent_ret annual_ret  
                        sum count sum                                        
ticker coint_ticker                                                          
AMZN   BAC             -0.6     4   2              7   -0.003512   0.832393  
       CI             19.29     4   4              7    0.015074   2.181792  
       HII             0.26     4   2              7     0.00032   1.016826  
       HSY             14.3     4   4              7    0.021401   3.016682  
       INVH            0.12     4   2              7    0.000879   1.046879  
...                     ...   ...  ..            ...         ...        ...  
XOM    GOOG           -8.96    10   4             29   -0.005348   0.934731  
       GOOGL          -9.53    10   3             29   -0.005759   0.929887  
       SJM            -9.01    10   3             29   -0.008082   0.902912  
       WSM           -62.99    10   0             29   -0.035934   0.630905  
       WY             -1.26    10   4             29   -0.004255   0.947748  

[177 rows x 18 columns]

In [3]:
tsla_cag = pd.read_csv(
    "./data/results/2025-03-28/ziweichen_5/price_actions/TSLA_CAG.csv"
)
tsla_cag

,date,ticker,day_name,is_holiday,av_rating,median_rating_excl,TSLA_close,CAG_close,action
0,2025-03-25,TSLA,Tuesday,False,2.764706,1.5,288.14,25.35,sell
1,2025-03-26,TSLA,Wednesday,False,3.163636,4.5,272.06,25.92,buy
2,2025-03-27,TSLA,Thursday,False,2.800000,2.0,273.13,26.46,sell
3,2025-03-28,TSLA,Friday,False,2.530612,1.0,NaN,NaN,sell


# [Future Works](#table-of-contents)

- Determine if news title suffice in sentiment analysis.
- Determine pvalue threshold for cointegration studies (Is lowest pvalue = highest profit)
- Abberation studies using profit/loss generated based on sentiment rating:

| S/N | Component            | Description                                                                                                               |
| --- | -------------------- | ------------------------------------------------------------------------------------------------------------------------- |
| 1.  | FinBert model        | &bull; Prosus AI, finbert-tone, FinBERT-FOMC, finbert-sentiment-analysis. <br>&bull; Currently tested on 'FinBERT-FOMC'.  |
| 2.  | News info            | &bull; news title, news content, both news title and content. <br>&bull; Currently tested on both news title and content. |
| 3.  | cointegration        | &bull; 1, 3 and 5 years. <br>&bull; Currently tested on 5 years.                                                          |
| 4.  | correlation analysis | &bull; Pearson (Linear correlation), Spearman & Kendall (monotonic correlation). <br>&bull; Currently not tested.         |
